In [ ]:
############################################################
# 0. Setup and Initial Checks + dtype fixes
############################################################
import numpy as np
import pandas as pd
from scipy.stats import describe
from sklearn.model_selection import GroupKFold, KFold

df = pd.read_csv("data.csv",low_memory=False)

# 0.1 Datetime conversion
if 'datadate' in df.columns:
    df['datadate'] = pd.to_datetime(df['datadate'], errors='coerce')

# 0.2 Numeric conversion for price columns (prcc_c, prcc_f)
for col in ['prcc_c', 'prcc_f']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# 0.3 Panel key types (safe even if some values are non-numeric)
for col in ['gvkey', 'fyear', 'ismod']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')

# 0.4 Categorical columns
for col in ['indfmt', 'datafmt', 'consol']:
    if col in df.columns:
        df[col] = df[col].astype('category')
    len(df)
print(df.dtypes)
print("Numeric columns:", len(df.select_dtypes(include=[np.number]).columns))
print("Categorical columns:", len(df.select_dtypes(include=['category']).columns))
print("Datetime columns:", len(df.select_dtypes(include=['datetime']).columns))

In [25]:
df.info()
df.describe()
missing_counts = df.isna().sum().sort_values(ascending=False)
missing_pct = (df.isna().mean() * 100).sort_values(ascending=False)
pd.DataFrame({"missing_count": missing_counts, "missing_pct": missing_pct})

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75005 entries, 0 to 75004
Data columns (total 89 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   gvkey     75005 non-null  Int64         
 1   datadate  75005 non-null  datetime64[ns]
 2   fyear     75005 non-null  Int64         
 3   indfmt    75005 non-null  category      
 4   datafmt   75005 non-null  category      
 5   consol    75005 non-null  category      
 6   ismod     75005 non-null  Int64         
 7   conm      75005 non-null  object        
 8   aco       68227 non-null  float64       
 9   act       64284 non-null  float64       
 10  ao        75000 non-null  float64       
 11  aoloch    74543 non-null  float64       
 12  ap        74917 non-null  float64       
 13  apalch    44634 non-null  float64       
 14  aqc       72193 non-null  float64       
 15  at        75005 non-null  float64       
 16  caps      71931 non-null  float64       
 17  capx      74

,missing_count,missing_pct
dlcch,33143,44.187721
apalch,30371,40.491967
txach,22791,30.385974
ivstch,19194,25.590294
recch,12589,16.784214
...,...,...
fyear,0,0.000000
datafmt,0,0.000000
ibadj,0,0.000000
ib,0,0.000000


In [26]:

############################################################
# 1.a Data Cleaning Duplicates
############################################################
# Remove duplicates (panel-unique if possible)
_before = len(df)
if all(c in df.columns for c in ['gvkey', 'fyear']):
    df = df.drop_duplicates(subset=['gvkey', 'fyear'], keep='last').reset_index(drop=True)
else:
    df = df.drop_duplicates().reset_index(drop=True)
_after = len(df)
print(f"Duplicates dropped: {_before - _after}")


Duplicates dropped: 0


In [ ]:
############################################################
# 2. Data Cleaning Outlier Statistics
#    - Only selected Compustat fields
#    - Robust flags: IQR rule + MAD-based z-score
############################################################
from scipy import stats


# 2.1 Requested variables
target_vars = [
    "sale", "ib", "ni", "xi", "xido", "dp", "xrd", "emp",
    "ebit", "ebitda", "ob", "fca", "irent"
]

# 2.1a Map ebit/ebitda if needed (Compustat mnemonics)
if "ebit" not in df.columns and "oiadp" in df.columns:
    df["ebit"] = pd.to_numeric(df["oiadp"], errors="coerce")
if "ebitda" not in df.columns and "oibdp" in df.columns:
    df["ebitda"] = pd.to_numeric(df["oibdp"], errors="coerce")

# 2.2 Keep only those present (but keep ALL target_vars in diagnostics output)
present_vars = [v for v in target_vars if v in df.columns]
missing_vars = [v for v in target_vars if v not in df.columns]

print(f"[Outliers] Present vars: {present_vars}")
print(f"[Outliers] Missing vars: {missing_vars}")

# 2.5 Compute per-variable stats + outlier counts (include missing target vars as diagnostic rows)
summaries = []
flag_cols = []

# Add explicit rows for missing target vars so they are not "dropped" from diagnostics
for v in missing_vars:
    summaries.append({
        "var": v,
        "n_nonmissing": 0,
        "missing_pct": 100.0,
        "note": "missing in df (not available for outlier flagging)"
    })

if not present_vars:
    summary_df = pd.DataFrame(summaries)

    if not summary_df.empty and "missing_pct" in summary_df.columns:
        summary_df = summary_df.sort_values(["missing_pct", "var"], ascending=[True, True])

    print("\n[Outliers] Per-variable statistics (requested vars only):")
    if summary_df.empty:
        print("(no requested variables found and none listed)")
    else:
        print(summary_df.to_string(index=False))
else:
    # 2.3 Coerce present vars to numeric + clean infinities
    for v in present_vars:
        df[v] = pd.to_numeric(df[v], errors="coerce")
    df[present_vars] = df[present_vars].replace([np.inf, -np.inf], np.nan)


    # 2.4 Robust flag functions
    def iqr_outlier_flags(s: pd.Series, k: float = 1.5) -> pd.Series:
        s = s.dropna()
        if len(s) < 30:
            return pd.Series(False, index=s.index)
        q1, q3 = s.quantile(0.25), s.quantile(0.75)
        iqr = q3 - q1
        if iqr == 0 or pd.isna(iqr):
            return pd.Series(False, index=s.index)
        lo, hi = q1 - k * iqr, q3 + k * iqr
        return (s < lo) | (s > hi)


    def mad_z_outlier_flags(s: pd.Series, z: float = 3.5) -> pd.Series:
        s = s.dropna()
        if len(s) < 30:
            return pd.Series(False, index=s.index)
        med = s.median()
        mad = (s - med).abs().median()
        if mad == 0 or pd.isna(mad):
            return pd.Series(False, index=s.index)
        mz = 0.6745 * (s - med) / mad
        return mz.abs() > z


    for v in present_vars:
        s = df[v]
        n = int(s.notna().sum())
        miss_pct = float(s.isna().mean() * 100)

        # skip truly unusable columns
        if n < 30 or s.std(skipna=True) == 0:
            summaries.append({
                "var": v,
                "n_nonmissing": n,
                "missing_pct": miss_pct,
                "note": "skipped (too few obs or zero variance)"
            })
            continue

        # IQR and MAD flags
        iqr_flags = pd.Series(False, index=df.index)
        mad_flags = pd.Series(False, index=df.index)

        _iqr = iqr_outlier_flags(s, k=1.5)
        _mad = mad_z_outlier_flags(s, z=3.5)

        iqr_flags.loc[_iqr.index] = _iqr.values
        mad_flags.loc[_mad.index] = _mad.values

        df[f"out_iqr_{v}"] = iqr_flags
        df[f"out_mad_{v}"] = mad_flags
        flag_cols.extend([f"out_iqr_{v}", f"out_mad_{v}"])

        # core descriptive stats (robust + tails)
        summaries.append({
            "var": v,
            "n_nonmissing": n,
            "missing_pct": miss_pct,
            "mean": float(s.mean(skipna=True)),
            "median": float(s.median(skipna=True)),
            "std": float(s.std(skipna=True)),
            "skew": float(s.skew(skipna=True)),
            "kurt": float(s.kurt(skipna=True)),
            "p1": float(s.quantile(0.01)),
            "p5": float(s.quantile(0.05)),
            "p25": float(s.quantile(0.25)),
            "p75": float(s.quantile(0.75)),
            "p95": float(s.quantile(0.95)),
            "p99": float(s.quantile(0.99)),
            "min": float(s.min(skipna=True)),
            "max": float(s.max(skipna=True)),
            "iqr_outliers": int(iqr_flags.sum()),
            "mad_outliers": int(mad_flags.sum())
        })

    summary_df = pd.DataFrame(summaries)

    # Sort: most complete first
    if "missing_pct" in summary_df.columns:
        summary_df = summary_df.sort_values(["missing_pct", "var"], ascending=[True, True])

    print("\n[Outliers] Per-variable statistics (requested vars only):")
    print(summary_df.to_string(index=False))

    # 2.6 Row-level “outlier burden” across present target vars only
    if flag_cols:
        df["outlier_flag_count_selvars"] = df[flag_cols].sum(axis=1)
        df["row_outlier_any_selvars"] = df["outlier_flag_count_selvars"] > 0
        df["row_outlier_many_selvars"] = df["outlier_flag_count_selvars"] >= 3

        print(f"\n[Outliers] Rows with ANY flag (selected vars): "
              f"{df['row_outlier_any_selvars'].sum()} "
              f"({df['row_outlier_any_selvars'].mean() * 100:.2f}%)")
        print(f"[Outliers] Rows with >=3 flags (selected vars): "
              f"{df['row_outlier_many_selvars'].sum()} "
              f"({df['row_outlier_many_selvars'].mean() * 100:.2f}%)")

        # Optional: show the top 20 most-flagged rows for audit
        audit_cols = [c for c in ["gvkey", "fyear", "datadate"] if c in df.columns] + present_vars
        top = df.sort_values("outlier_flag_count_selvars", ascending=False).head(20)
        print("\n[Outliers] Top 20 rows by outlier_flag_count_selvars (audit view):")
        print(top[audit_cols + ["outlier_flag_count_selvars"]].to_string(index=False))
